# **[Nerdle Mini](https://mini.nerdlegame.com/) "*Random*" Simulations**

This notebook performs **1,000** "*random*" simulations for each one of the **206** possible solutions of the game, totalling **206,000** simulations. This simulations aren't completely random, the strategy work as follows:

+ After each attempt, the equations are filtered based on the hits and misses of each element, so, the next attempt is going to be an equation randomly selected from the filtered ones.
+ The first **2** attempts apply a condition to pick only available equations that have the maximum number of unique elements, maximizing the chances of getting good results.
+ It simulates "*hard mode*" where the next attempt always uses the correct and misplaced elements from the current attempt.

The output file utilizes the following chars to represent the hits and misses:

+ **c** : Correctly placed element (🟩 - Green)
+ **m** : Misplaced element (🟪 - Purple)
+ _ : Incorrect element (⬛ - Black)

# Imports and Constants

In [ ]:
from multiprocessing import Pool
from tqdm import tqdm
import pandas as pd


# Game parameters
NUMBER_OF_ATTEMPTS = 6
NUMBER_OF_ELEMENTS = 6

# Elements
ELEMENT_EMPTY = " "
ELEMENT_CORRECT = "c"
ELEMENT_MISPLACED = "m"
ELEMENT_INCORRECT = "_"
ELEMENTS = list("1234568790+-*/=")

# Attempts
ATTEMPT_CORRECT = ELEMENT_CORRECT * NUMBER_OF_ELEMENTS
ATTEMPT_INCORRECT = ELEMENT_INCORRECT * NUMBER_OF_ELEMENTS
EMPTY_GAME_SIMULATION = [ELEMENT_EMPTY * NUMBER_OF_ELEMENTS] * (NUMBER_OF_ATTEMPTS * 2)


# CSV header
CSV_HEADER = ["solution"]
for i in range(NUMBER_OF_ATTEMPTS):
    CSV_HEADER.append(f"attempt_{i}")
    CSV_HEADER.append(f"hits_{i}")

In [ ]:
# Simulation parameters

SIMS_COUNT = 1_000
UNIQUE_ATTEMPTS = 2

FILEPATH_IN = f"./data/0.raw/equations_mini_nerdle.csv"
FILEPATH_OUT = f"./data/2.simulations/mini_nerdle/simulations_random.csv"

# Load data

*(And setting the index)*

In [ ]:
df_equations = pd.read_csv(FILEPATH_IN)
df_equations.index = df_equations["equation"]
df_equations

# Pre processing

In [ ]:
# Init a dictionary to store all temporary DataFrames
dfs = {k: pd.DataFrame() for k in ["positions", "unique", "elements"]}


# Spliting positions int o columns
for i in range(NUMBER_OF_ELEMENTS):
    dfs["positions"][f"p{i}"] = df_equations["equation"].str[i].astype("category")

# Counting the number of unique elements per equation
dfs["unique"]["count"] = df_equations["equation"].apply(lambda x: len(set(x))).astype("uint8")

# Counting the amount of each element in each equation
for element in ELEMENTS:
    dfs["elements"][element] = df_equations["equation"].apply(lambda x: x.count(element)).astype("uint8")


# Concatenating the DataFrames into a single multi-indexed one
df_equations = pd.concat(dfs, axis=1)
df_equations

# **Simulation functions**

### Compute the hits of the attempt

In [ ]:
def compute_hits(solution: str, attempt: str):

    # Init counter
    counter = {}

    # Count the number of each unique element in the attempt
    for element in set(attempt):
        counter[element] = solution.count(element)

    # Init hits
    hits = list(ATTEMPT_INCORRECT)

    # Compute 'correct' hits
    for i in range(NUMBER_OF_ELEMENTS):
        if attempt[i] == solution[i]:
            counter[attempt[i]] -= 1
            hits[i] = ELEMENT_CORRECT

    # Compute 'misplaced' hits
    for i in range(NUMBER_OF_ELEMENTS):
        if counter[attempt[i]] > 0 and hits[i] == ELEMENT_INCORRECT:
            counter[attempt[i]] -= 1
            hits[i] = ELEMENT_MISPLACED

    # Return hits
    return "".join(hits)

In [ ]:
# Perform tests

TESTS = [
    (("1+9=10", "8+9=17"), "_cccc_"),
    (("5*8=40", "14/7=2"), "_m__m_"),
    (("14/7=2", "10/5=2"), "c_c_cc"),
    (("13-9=4", "12-3=9"), "c_cmcm"),
    (("36/6=6", "10-3=7"), "___mc_"),
]

for test, result in TESTS:
    assert compute_hits(*test) == result
else:
    print("All 'compute_hits()' tests passed")

### Count the number of each element in the attempt based on the attempt hits

In [ ]:
def count_elements(attempt: str, hits: str):

    # Init counter
    counter = {element: {"min": 0, "max": 0} for element in set(attempt)}

    # Count the sum of correct and misplaced elements as well as the number of incorrect elements
    for i in range(NUMBER_OF_ELEMENTS):
        if hits[i] in (ELEMENT_MISPLACED, ELEMENT_CORRECT):
            counter[attempt[i]]["min"] += 1

        else:
            counter[attempt[i]]["max"] += 1

    # Set the counter to the min and max possible elements in the solution
    for element in counter:
        counts = counter[element]

        if counts["min"] == 0:
            counts["max"] = 0

        elif counts["max"] == 0:
            counts["max"] = NUMBER_OF_ELEMENTS

        elif counts["max"] != 0:
            counts["max"] = counts["min"]

    # Return the counter
    return counter

In [ ]:
# Perform tests

TESTS = [
    (("1+9=10", "_cccc_"), {"0": {"min": 0, "max": 0}, "1": {"min": 1,"max": 1}, "+": {"min": 1, "max": 6}, "9": {"min": 1, "max": 6}, "=": {"min": 1, "max": 6}}),
    (("5*8=40", "_m__m_"), {"8": {"min": 0, "max": 0}, "*": {"min": 1,"max": 6}, "4": {"min": 1, "max": 6}, "0": {"min": 0, "max": 0}, "5": {"min": 0, "max": 0}, "=": {"min": 0, "max": 0}}),
    (("14/7=2", "c_c_cc"), {"4": {"min": 0, "max": 0}, "2": {"min": 1,"max": 6}, "1": {"min": 1, "max": 6}, "7": {"min": 0, "max": 0}, "/": {"min": 1, "max": 6}, "=": {"min": 1, "max": 6}}),
    (("13-9=4", "c_cmcm"), {"3": {"min": 0, "max": 0}, "-": {"min": 1,"max": 6}, "4": {"min": 1, "max": 6}, "1": {"min": 1, "max": 6}, "9": {"min": 1, "max": 6}, "=": {"min": 1, "max": 6}}),
    (("36/6=6", "___mc_"), {"6": {"min": 1, "max": 1}, "3": {"min": 0,"max": 0}, "/": {"min": 0, "max": 0}, "=": {"min": 1, "max": 6}})
]

for test, result in TESTS:
    assert count_elements(*test) == result
else:
    print("All 'count_elements()' tests passed")

### Filter the candidate equations based on the attempt hits and attempt counter

In [ ]:
def filter_equations(equations: pd.DataFrame, equation: str, hits: str, counter: dict):

    # Filter based on the element counts
    for element in counter:
        equations = equations[equations["elements"][element] >= counter[element]["min"]]
        equations = equations[equations["elements"][element] <= counter[element]["max"]]

    # Filter based on the attempt hits positions
    for i in range(NUMBER_OF_ELEMENTS):
        if hits[i] == ELEMENT_CORRECT:
            equations = equations[equations["positions"][f"p{i}"] == equation[i]]

        else:
            equations = equations[equations["positions"][f"p{i}"] != equation[i]]
    
    # Return the filtered equations
    return equations

### Simulate all of the attempts of a game

In [ ]:
def play_game(equations: pd.DataFrame, solution: str, unique_attempts: int,):

    # Init output simulation data
    simulation = [solution] + EMPTY_GAME_SIMULATION

    # Iterate over the attempts
    for i in range(NUMBER_OF_ATTEMPTS):

        # Get equations with the most unique elements based on the number of unique attempts passed
        if i < unique_attempts:
            temp = equations[equations[("unique", "count")] == equations[("unique", "count")].max()]
        else:
            temp = equations

        # Get a random equation and compute hits
        equation = temp.sample(1).iloc[0].name
        hits = compute_hits(solution, equation)

        # Store values on the simulation data
        simulation[(2 * i) + 1] = equation
        simulation[(2 * i) + 2] = hits

        # Break if the solution is found
        if hits == ATTEMPT_CORRECT:
            break
        
        # Filter equations based on the attempt hits
        counter = count_elements(equation, hits)
        equations = filter_equations(equations, equation, hits, counter)

    # Return simulation data
    return simulation

### Simulate a game for every solution

In [ ]:
def simulate_all_solutions(equations: pd.DataFrame):
    
    # Init simulations data
    simulations = []

    # Iterate over every solution
    for solution in equations.index:

        # Run and append the simulation data
        simulations.append(play_game(equations, solution, UNIQUE_ATTEMPTS))
    
    # Append the simulation data to the CSV file
    with open(FILEPATH_OUT, "a", newline="") as f:
        pd.DataFrame(simulations, columns=CSV_HEADER).to_csv(f, header=False, index=False)

# Run simulations

In [ ]:
# Init output file
with open(FILEPATH_OUT, "w", newline="") as f:
    pd.DataFrame(columns=CSV_HEADER).to_csv(f, index=False)

# Starting the multiprocessing Pool
with Pool() as p:
    list(tqdm(p.imap(simulate_all_solutions, [df_equations for _ in range(SIMS_COUNT)]), total=SIMS_COUNT))